<a href="https://colab.research.google.com/github/iHuzama/AI605-NLP/blob/main/KAIST_AI605_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI605 Assignment 2: Retrieval
TA in charge: Miyoung Ko (miyoungko@kaist.ac.kr)

**Due Date:** Apr 26 (Tue) 11:00pm, 2022

## Your Submission
If you are a KAIST student, you will submit your assignment via [KLMS](https://klms.kaist.ac.kr). If you are a NAVER student, you will submit via [Google Form](https://forms.gle/FSng5HUwtQinTFAU8). 

You need to submit both (1) a PDF of this notebook, and (2) a link to CoLab for execution (.ipynb file is also allowed).

Use in-line LaTeX (see below) for mathematical expressions. Collaboration among students is allowed but it is not a group assignment so make sure your answer and code are your own. Make sure to mention your collaborators in your assignment with their names and their student ids.

## Grading
The entire assignment is out of 20 points. You can obtain up to 2 bonus points (i.e. max score is 22 points). For every late day, your grade will be deducted by 2 points (KAIST students only). You can use one of your no-penalty late days (7 days in total). Make sure to mention this in your submission. You will receive a grade of zero if you submit after 7 days.


## Environment
You will need Python 3.7+ and PyTorch 1.9+, which are already available on Colab:

In [1]:
from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.13
torch 1.10.0+cu111


You will use SQuAD, a classic machine reading comprehension dataset, in this assignment. Note that while this is an MRC dataset, we will also use it for retrieval by trying to find the correct document corresponding to the question among all the documents in the **validation** data. 

In [2]:
!pip install -q datasets

     |████████████████████████████████| 325 kB 10.2 MB/s 
     |████████████████████████████████| 1.1 MB 37.5 MB/s 
     |████████████████████████████████| 136 kB 57.5 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 212 kB 43.7 MB/s 
     |████████████████████████████████| 127 kB 52.0 MB/s 
     |████████████████████████████████| 94 kB 1.2 MB/s 
     |████████████████████████████████| 144 kB 15.0 MB/s 
     |████████████████████████████████| 271 kB 51.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
from datasets import load_dataset
from pprint import pprint

squad_dataset = load_dataset('squad')
pprint(squad_dataset['train'][0]) # 'context' contains the document

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the '
            "Main Building's gold dome is a golden statue of the Virgin Mary. "
            'Immediately in front of the Main Building and facing it, is a '
            'copper statue of Christ with arms upraised with the legend '
            '"Venite Ad Me Omnes". Next to the Main Building is the Basilica '
            'of the Sacred Heart. Immediately behind the basilica is the '
            'Grotto, a Marian place of prayer and reflection. It is a replica '
            'of the grotto at Lourdes, France where the Virgin Mary reputedly '
            'appeared to Saint Bernadette Soubirous in 1858. At the end of the '
            'main drive (and in a direct line that connects through 3 statues '
            'and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did t

## 1. Measuring Similarity
We discussed in Lecture 04 that there are several ways to measure similarity between two vectors, such as L2 (Euclidean) distance, L1 (Manhattan) distance, inner product, and cosine distance. Here, only L1 and L2 (and angular distance) are *metric* (see *Definition* at https://en.wikipedia.org/wiki/Metric_(mathematics)).

> **Problem 1.1** *(2 points)* Using the definition of metric above, prove that L1 distance is a metric.

> **Problem 1.2** *(2 points)* Prove that negative inner product is NOT a metric.

> **Problem 1.3** *(2 points)* Prove that cosine distance (1 - cosine similarity) is NOT a metric.

> **Problem 1.4 (bonus)** *(2 points)* Given a model that can perform nearest neighbor search in L2 space, can you modify your query and your key vectors to perform maximum inner product search? (Hint: Recall the difference between MIPS and L2 NNS in Lecture 04. Can you modify key vectors so that the difference becomes 0?)

## 2. Sparse Search



We first create an abstract class for performing similarity search as follows (`raise NotImplementedError()` means you have to override these methods when you subclass the class):

In [4]:
class SimilaritySearch(object):
  def __init__(self):
    raise NotImplementedError()

  def train(self, documents: list):
    raise NotImplementedError()

  #Add documents (a list of text)
  def add(self, documents: list):
    raise NotImplementedError()

  #Returns the indices of top-k documents among the added documents
  #that are most similar to the input query 
  def search(self, query: str, k: int) -> list:
    raise NotImplementedError()


You will use the same space-based tokenizer that you used in Assignment 1, with lowercasing to make it case insensitive.

> **Problem 2.1** *(2 points)* We will first start with Bag of Words that we discussed in Lecture 08. Using the definition in the class (don't worry about the exact definition though), implement `BagOfWords` class that subclasses `SimilaritySearch` class.

In [5]:
import torchtext
from torchtext.data import get_tokenizer
import numpy as np
import re
import collections

tokenizer = get_tokenizer("basic_english") 

dataSet = squad_dataset['validation']
contexts = dataSet['context']
uniqueContexts = list(set(squad_dataset['validation']['context']))

a = contexts.index(uniqueContexts[1])

In [12]:
class BagOfWords(SimilaritySearch):
  def __init__(self):
    self.word2id = {}
    self.BOWs = None

  #Add documents (a list of text)
  def add(self, documents: list):
    string = ' '.join([str(elem).strip('\n') for elem in documents])
    vocab = list(set(tokenizer(string)))
    self.word2id = {word: id_ for id_, word in enumerate(vocab)}
    self.BOWs = torch.zeros([len(documents), len(self.word2id)])
    for documentID in range(len(documents)):
      document = documents[documentID]
      words = tokenizer(document)
      for word in words:
        self.BOWs[documentID, self.word2id[word]] = 1

  #Returns the indices of top-k documents among the added documents
  #that are most similar to the input query 
  def search(self, query: str, k: int) -> list:
    words = list(set(tokenizer(query)))
    hotVector = torch.zeros([1, len(self.word2id)])
    for i in words:
      if i in self.word2id.keys():
        hotVector[0, self.word2id[i]] = 1
    simili = torch.nn.CosineSimilarity()(self.BOWs, hotVector)
    return torch.topk(simili, k)

> **Problem 2.2** *(2 points)* Using the definition in Lecture 08 (don't worry about the exact definition though), implement `TFIDF` class that subclasses `BagOfWords` class. Use natural log (instead of log with base 10).

In [24]:
class TFIDF(BagOfWords):
  def __init__(self):
    self.word2id = {}
    self.BOW = None
    self.tfidf = None
    self.idf = None

  #Add documents (a list of text)
  def add(self, documents: list):
    string = ' '.join([str(elem).strip('\n') for elem in documents])
    vocab = list(set(tokenizer(string)))

    self.word2id = {word: id_ for id_, word in enumerate(vocab)}
    self.BOW = torch.zeros([len(documents), len(self.word2id)])

    for documentID in range(len(documents)):
      document =documents[documentID]
      words = tokenizer(document)
      for word in words:
        self.BOW[documentID, self.word2id[word]] = words.count(word)

  #Returns the indices of top-k documents among the added documents
  #that are most similar to the input query 
  def search(self, query: str, k: int) -> list:
    words = list(set(tokenizer(query)))
    hotVector = torch.zeros([1, len(self.word2id)])

    query_tf = collections.Counter(words)
    query_tfidf = torch.zeros([1, len(self.word2id)])

    for word in words:
      if word in self.word2id.keys():
        query_tfidf[0, self.word2id[word]] = query_tf[word] * self.idf[self.word2id[word]]

    simili = torch.nn.CosineSimilarity()(self.tfidf, query_tfidf)
    return torch.topk(simili, k)


  def train(self, documents: list):
    self.add(documents)
    self.documents_tf = self.BOW
    df = (self.documents_tf > 0)
    df = df.sum(0)
    self.idf = np.log(self.documents_tf.shape[0]) - np.log(df)
    self.tfidf = torch.zeros([len(documents), len(self.word2id)])
    for term in self.word2id.keys():
      self.tfidf[:, self.word2id[term]] = self.BOW[:, self.word2id[term]] * self.idf[self.word2id[term]]

> **Problem 2.3** *(2 points)* Use `TFIDF` to masure the recall rate of the correct document when 10 documents (contexts) are retrieved (this is called **Recall@10**) in SQuAD **validation** set.

In [25]:
BOWSimilarity = BagOfWords()
BOWSimilarity.add(uniqueContexts)

TFIDFSimilarity = TFIDF()
TFIDFSimilarity.train(uniqueContexts)

In [ ]:
bowRecall = 0
tfidfRecall = 0
for i in range(len(dataSet)):
  originalTitle = dataSet[i]['title']

  tfIDFsimili = TFIDFSimilarity.search(dataSet[i]['question'], 10)
  bowSimili = BOWSimilarity.search(dataSet[i]['question'], 10)
  for j in range(10):
    mappedIndex = contexts.index(uniqueContexts[tfIDFsimili[j]])
    if originalTitle == dataSet[mappedIndex]['title']:
      tfidfRecall += 1
    
    mappedIndex = contexts.index(uniqueContexts[bowSimili[j]])
    if originalTitle == dataSet[mappedIndex]['title']:
      bowRecall += 1

bowRecall /= (10*len(dataSet))
tfidfRecall /= (10*len(dataSet))

  
    
  

In [35]:
contexts.index(uniqueContexts[1, 2])

TypeError: ignored

## 3. Dense Search

To obtain the embedding of each document and query, you will use pretrained word embeddings. Recall that most word embeddings are trained in a self-supervised way from a large text corpus. Here, we will use BERT word embeddings, which are also self-supervised. You will compute the document's embedding by simply averaging the embeddings of all words in the document (same for the query), and then normalizing it. This way, inner product effectively becomes cosine similarity.

BERT word embeddings can be easily obtained by using `transformers` library by Hugging Face. First, install the library: 

In [ ]:
!pip install -q transformers

You will then download the necessary tokenizer and the model.

In [ ]:
from transformers import AutoTokenizer, AutoModel

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Note that the tokenizer behaves a little differently from what you have done so far. It is a subword tokenizer and it inserts special tokens at the first and at the last, which should be ignored when you are computing the average.

In [ ]:
text = "Hello KAIST!"
tokens = tokenizer.tokenize(text)
input_ = tokenizer(text)
input_tensor = tokenizer(text, return_tensors='pt')

print(tokens) # ['hello', 'kai', '##st', '!'] Note that (1) ## indicates subword, and (2) all characters are lowercased.
print(input_['input_ids']) # [101, 7592, 11928, 3367, 999, 102], where the first and the last tokens are special tokens
print(input_tensor['input_ids']) # same as line 7 but in PyTorch tensor
print(tokenizer.convert_ids_to_tokens(input_['input_ids'])) # you will verify that the first and the last tokens are special tokens

The model contains not only the word embeddings but also the BERT model parameters. Here, you will only use embeddings (you will use the full model in Assignment 4).

In [ ]:
output = model.embeddings(input_tensor['input_ids'])
print(output.size()) # [1, 6, 768], where the first dim is batch size, second dim is number of tokens, and third is hidden size

> **Problem 3.1** *(2 points)* Implement `BERTEmbeddingSearch` class that subclasses `SimilaritySearch` class, using PyTorch's tensor native operation for the dense search. 

> **Problem 3.2** *(2 points)* Use `BERTEmbeddingSearch` to measure the recall at 10 for SQuAD validation dataset. How does it compare to TFIDF?

> **Problem 3.3** *(2 points)* Implement `BERTEmbeddingFaiss` that subclasses `SimilaritySearch` and uses Faiss `IndexFlatIP` instead of PyTorch native tensor operation for search. Refer to the Faiss wiki (https://github.com/facebookresearch/faiss/wiki/Getting-started) for instructions.

> **Problem 3.4** *(2 points)* Compare the speed between `BERTEmbeddingSearch` and `BERTEmbeddingFaiss` on SQuAD. To make the measurement accurate, perform search many times (at least more than 1000) and take the average.